# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 16 2022 12:00AM,239692,10,8149158,"Citieffe, Inc.",Released
1,Jun 16 2022 12:00AM,239694,10,8149228,"Herbivore Botanicals, LLC.",Released
2,Jun 15 2022 3:41PM,239690,19,ADV80003368,"AdvaGen Pharma, Ltd",Released
3,Jun 15 2022 3:41PM,239690,19,ADV80003369,"AdvaGen Pharma, Ltd",Released
4,Jun 15 2022 3:41PM,239690,19,ADV80003370,"AdvaGen Pharma, Ltd",Released
5,Jun 15 2022 3:41PM,239690,19,ADV80003371,"AdvaGen Pharma, Ltd",Released
6,Jun 15 2022 3:41PM,239690,19,ADV80003372,"AdvaGen Pharma, Ltd",Released
7,Jun 15 2022 3:41PM,239690,19,ADV80003373,"AdvaGen Pharma, Ltd",Released
8,Jun 15 2022 3:41PM,239690,19,ADV80003374,"AdvaGen Pharma, Ltd",Released
9,Jun 15 2022 3:41PM,239690,19,ADV80003375,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,239689,Released,1
31,239690,Released,29
32,239691,Released,1
33,239692,Released,1
34,239694,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239689,NaN,NaN,1.0
239690,NaN,NaN,29.0
239691,NaN,NaN,1.0
239692,NaN,NaN,1.0
239694,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239614,0.0,0.0,1.0
239629,0.0,0.0,1.0
239641,21.0,3.0,2.0
239642,0.0,0.0,1.0
239643,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239614,0,0,1
239629,0,0,1
239641,21,3,2
239642,0,0,1
239643,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239614,0,0,1
1,239629,0,0,1
2,239641,21,3,2
3,239642,0,0,1
4,239643,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239614,,,1
1,239629,,,1
2,239641,21,3,2
3,239642,,,1
4,239643,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 16 2022 12:00AM,239692,10,"Citieffe, Inc."
1,Jun 16 2022 12:00AM,239694,10,"Herbivore Botanicals, LLC."
2,Jun 15 2022 3:41PM,239690,19,"AdvaGen Pharma, Ltd"
31,Jun 15 2022 3:40PM,239691,20,Sartorius Stedim North America
32,Jun 15 2022 3:28PM,239687,12,Hush Hush
35,Jun 15 2022 3:26PM,239689,22,"NBTY Global, Inc."
36,Jun 15 2022 3:19PM,239686,18,"ClearSpec, LLC"
37,Jun 15 2022 3:13PM,239685,10,Emerginnova
39,Jun 15 2022 3:06PM,239684,10,"Citieffe, Inc."
41,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 16 2022 12:00AM,239692,10,"Citieffe, Inc.",,,1
1,Jun 16 2022 12:00AM,239694,10,"Herbivore Botanicals, LLC.",,,1
2,Jun 15 2022 3:41PM,239690,19,"AdvaGen Pharma, Ltd",,,29
3,Jun 15 2022 3:40PM,239691,20,Sartorius Stedim North America,,,1
4,Jun 15 2022 3:28PM,239687,12,Hush Hush,,,3
5,Jun 15 2022 3:26PM,239689,22,"NBTY Global, Inc.",,,1
6,Jun 15 2022 3:19PM,239686,18,"ClearSpec, LLC",,,1
7,Jun 15 2022 3:13PM,239685,10,Emerginnova,,,2
8,Jun 15 2022 3:06PM,239684,10,"Citieffe, Inc.",,,2
9,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 12:00AM,239692,10,"Citieffe, Inc.",1,,
1,Jun 16 2022 12:00AM,239694,10,"Herbivore Botanicals, LLC.",1,,
2,Jun 15 2022 3:41PM,239690,19,"AdvaGen Pharma, Ltd",29,,
3,Jun 15 2022 3:40PM,239691,20,Sartorius Stedim North America,1,,
4,Jun 15 2022 3:28PM,239687,12,Hush Hush,3,,
5,Jun 15 2022 3:26PM,239689,22,"NBTY Global, Inc.",1,,
6,Jun 15 2022 3:19PM,239686,18,"ClearSpec, LLC",1,,
7,Jun 15 2022 3:13PM,239685,10,Emerginnova,2,,
8,Jun 15 2022 3:06PM,239684,10,"Citieffe, Inc.",2,,
9,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 12:00AM,239692,10,"Citieffe, Inc.",1,,
1,Jun 16 2022 12:00AM,239694,10,"Herbivore Botanicals, LLC.",1,,
2,Jun 15 2022 3:41PM,239690,19,"AdvaGen Pharma, Ltd",29,,
3,Jun 15 2022 3:40PM,239691,20,Sartorius Stedim North America,1,,
4,Jun 15 2022 3:28PM,239687,12,Hush Hush,3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 12:00AM,239692,10,"Citieffe, Inc.",1,NaN,NaN
1,Jun 16 2022 12:00AM,239694,10,"Herbivore Botanicals, LLC.",1,NaN,NaN
2,Jun 15 2022 3:41PM,239690,19,"AdvaGen Pharma, Ltd",29,NaN,NaN
3,Jun 15 2022 3:40PM,239691,20,Sartorius Stedim North America,1,NaN,NaN
4,Jun 15 2022 3:28PM,239687,12,Hush Hush,3,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 12:00AM,239692,10,"Citieffe, Inc.",1,0.0,0.0
1,Jun 16 2022 12:00AM,239694,10,"Herbivore Botanicals, LLC.",1,0.0,0.0
2,Jun 15 2022 3:41PM,239690,19,"AdvaGen Pharma, Ltd",29,0.0,0.0
3,Jun 15 2022 3:40PM,239691,20,Sartorius Stedim North America,1,0.0,0.0
4,Jun 15 2022 3:28PM,239687,12,Hush Hush,3,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2396773,68,0.0,0.0
12,239687,3,0.0,0.0
15,239648,1,1.0,0.0
18,718957,3,0.0,0.0
19,958656,45,0.0,0.0
20,1917344,38,75.0,21.0
22,958666,4,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2396773,68,0.0,0.0
1,12,239687,3,0.0,0.0
2,15,239648,1,1.0,0.0
3,18,718957,3,0.0,0.0
4,19,958656,45,0.0,0.0
5,20,1917344,38,75.0,21.0
6,22,958666,4,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,68,0.0,0.0
1,12,3,0.0,0.0
2,15,1,1.0,0.0
3,18,3,0.0,0.0
4,19,45,0.0,0.0
5,20,38,75.0,21.0
6,22,4,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,68.0
1,12,Released,3.0
2,15,Released,1.0
3,18,Released,3.0
4,19,Released,45.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,21.0,0.0
Executing,0.0,0.0,1.0,0.0,0.0,75.0,0.0
Released,68.0,3.0,1.0,3.0,45.0,38.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,0.0,21.0,0.0
1,Executing,0.0,0.0,1.0,0.0,0.0,75.0,0.0
2,Released,68.0,3.0,1.0,3.0,45.0,38.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,0.0,21.0,0.0
1,Executing,0.0,0.0,1.0,0.0,0.0,75.0,0.0
2,Released,68.0,3.0,1.0,3.0,45.0,38.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()